<a href="https://colab.research.google.com/github/analyticsariel/projects/blob/master/Zillow_Real_Estate_Data_Quandl_API_Get_Price_to_Rent_Ratio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Zillow Real Estate Data Quandl API: Get Price to Rent Ratio

## Overview
| Detail Tag            | Information                                                                                        |
|-----------------------|----------------------------------------------------------------------------------------------------|
| Originally Created By | Ariel Herrera arielherrera@analyticsariel.com                                                      |
| External References   | Zillow Real Estate Data Quandl Nasdaq API |
| Input Datasets        | Quandl API key                                                                                    |
| Output Datasets       | Price to Rent Ratio Map |
| Input Data Source     | None |
| Output Data Source    | Plotly Map |

## History
| Date         | Developed By  | Reason                                                |
|--------------|---------------|-------------------------------------------------------|
| 15th Feb 2022 | Ariel Herrera | Create notebook. |

## Getting Started
1. Copy this notebook -> File -> Save a Copy in Drive
2. Request [Nasdaq Data API Key](https://data.nasdaq.com/sign-up)

## Useful Resources
- [Google Collab Cheat Sheet](https://towardsdatascience.com/cheat-sheet-for-google-colab-63853778c093)
- [Nasdaq Database - Zillow Documentation](https://data.nasdaq.com/databases/ZILLOW/data)
- [Zillow Housing Data](https://www.zillow.com/research/data/)
- [Medium Post by Sajid](https://towardsdatascience.com/python-i-have-tested-quandl-api-and-how-to-get-real-estates-economics-data-in-one-line-of-code-a13806ca9bb)
- [Price to Rent Ratio by Stessa](https://www.stessa.com/blog/price-to-rent-ratio/)
- [Plotly Express](https://plotly.com/python/basic-charts/)

## <font color="blue">Install Packages</font>

In [8]:
!pip install quandl -q # api
!pip install pgeocode -q # geo coordinates

## <font color="blue">Imports</font>

In [9]:
from google.colab import drive, files # specific to Google Colab
import re
import pgeocode

# api
import quandl

# data manipulation
import warnings
import numpy as np
import pandas as pd

# visual
import plotly.express as px

warnings.filterwarnings('ignore') # remove warnings
pd.set_option('display.max_colwidth', 0) # view all columns
nomi = pgeocode.Nominatim('us') # USA coordinates

## <font color="blue">Functions</font>

In [10]:
def check_state_in_str(search_str):
  search_str_list = [x.strip() for x in search_str.split(';')]
  for x in search_str_list:
    if x in states:
      return x

def check_county_in_str(search_str):
  search_str_list = [x.strip() for x in search_str.split(';')]
  for x in search_str_list:
    if 'county' in x.lower():
      return x

def check_city_in_str(search_str):
  search_str_list = [x.strip() for x in search_str.split(';')]
  # if len is 1 then only zip code, if 5 then has all objects in str
  if len(search_str_list) == 1:
    return np.nan

  # else city will only occur at the end of the string
  # TODO: full proof way would be searching for metro name from FRED database
  if 'county' not in search_str_list[-1].lower():
    return search_str_list[-1]

def check_metro_in_str(search_str):
  search_str_list = [x.strip() for x in search_str.split(';')]
  # if len is 1 then only zip code, if 5 then has all objects in str
  if len(search_str_list) <= 3: # exploration: no metro in 3 objs or less
    return np.nan

  # else third positions should have metro
  if 'county' not in search_str_list[2].lower():
    return search_str_list[2]

## <font color="blue">Locals & Constants</font>

In [11]:
############
# OPTIONAL #
############

# mount drive
drive.mount('/content/drive', force_remount=False)

# data location
file_dir = '/content/drive/My Drive/Colab Data/input/' # optional

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
# read in api key file
df_api_keys = pd.read_csv(file_dir + 'api_keys.csv') # if you have your key stored in a csv file

# get keys
quandl_api_key = df_api_keys.loc[df_api_keys['API'] =='quandl']['KEY'].iloc[0] # replace this with your own key
mapbox_key = df_api_keys.loc[df_api_keys['API'] =='mapbox']['KEY'].iloc[0] # replace this with your own key

# enter your key here
quandl.ApiConfig.api_key = quandl_api_key

## <font color="blue">Data</font>

### <font color="green">1. Indicators</font>
[Nasdaq Indicators Documentation](https://data.nasdaq.com/tables/ZILLOW/ZILLOW-INDICATORS)

In [13]:
# get indicators
df_ind = quandl.get_table("ZILLOW/INDICATORS", paginate=True)
print('Num of indicators:', len(df_ind), '\n')

# view count of indicators by category
df_ind.groupby(['category'])['indicator_id'].count()

Num of indicators: 56 



category
Home values            10
Inventory and sales    44
Rentals                2 
Name: indicator_id, dtype: int64

In [14]:
# indicators for 'Home Values' category
df_ind.loc[df_ind['category'] == 'Home values']

,indicator_id,indicator,category
None,,,
0,ZSFH,ZHVI Single-Family Homes Time Series ($),Home values
1,ZCON,ZHVI Condo/Co-op Time Series ($),Home values
2,ZATT,ZHVI All Homes- Top Tier Time Series ($),Home values
3,ZALL,"ZHVI All Homes (SFR, Condo/Co-op) Time Series ($)",Home values
4,ZABT,ZHVI All Homes- Bottom Tier Time Series ($),Home values
5,Z5BR,ZHVI 5+ Bedroom Time Series ($),Home values
6,Z4BR,ZHVI 4-Bedroom Time Series ($),Home values
7,Z3BR,ZHVI 3-Bedroom Time Series ($),Home values
8,Z2BR,ZHVI 2-Bedroom Time Series ($),Home values


In [15]:
# indicators for 'Rentals' category
df_ind.loc[df_ind['category'] == 'Rentals']

,indicator_id,indicator,category
None,,,
22,RSSA,"ZORI (Smoothed, Seasonally Adjusted): All Homes Plus Multifamily Time Series ($)",Rentals
23,RSNA,ZORI (Smoothed): All Homes Plus Multifamily Time Series ($),Rentals


### <font color="green">2. Regions</font>
[Nasdaq Regions Documentation](https://data.nasdaq.com/tables/ZILLOW/ZILLOW-REGIONS)

In [16]:
# get regions
df_regions = quandl.get_table("ZILLOW/REGIONS", paginate=True)
print('Num of regions:', len(df_regions))
print('Region types:', df_regions['region_type'].unique(), '\n')
df_regions.head()

Num of regions: 79249
Region types: ['zip' 'city' 'county' 'neigh' 'state' 'metro'] 



,region_id,region_type,region
None,,,
0,99999,zip,98847; WA; Wenatchee; Chelan County; Peshastin
1,99998,zip,98846; WA; Okanogan County; Pateros
2,99997,zip,98845; WA; Wenatchee; Douglas County; Palisades
3,99996,zip,98844; WA; Okanogan County; Oroville
4,99995,zip,98843; WA; Wenatchee; Douglas County; Orondo


In [17]:
##########################
# Normalize Regions Data #
##########################

# all states
states = ['IA', 'KS', 'UT', 'VA', 'NC', 'NE', 'SD', 'AL', 'ID', 'FM', 'DE', 'AK', 'CT', 'PR', 'NM', 'MS', 'PW', 'CO', 'NJ', 'FL', 'MN', 'VI', 'NV', 'AZ', 'WI', 'ND', 'PA', 'OK', 'KY', 'RI', 'NH', 'MO', 'ME', 'VT', 'GA', 'GU', 'AS', 'NY', 'CA', 'HI', 'IL', 'TN', 'MA', 'OH', 'MD', 'MI', 'WY', 'WA', 'OR', 'MH', 'SC', 'IN', 'LA', 'MP', 'DC', 'MT', 'AR', 'WV', 'TX']

# filter on region type
df_regions_zip = df_regions.loc[df_regions['region_type'] == 'zip']

# get all regions content
df_regions_zip['region_str_len'] = df_regions_zip.apply(lambda x: len(x['region'].split(';')), axis=1)
df_regions_zip['zip_code'] = df_regions_zip.apply(lambda x: re.search('(\d{5})', x['region']).group(), axis=1)
df_regions_zip['state'] = df_regions_zip.apply(lambda x: check_state_in_str(x['region']), axis=1)
df_regions_zip['county'] = df_regions_zip.apply(lambda x: check_county_in_str(x['region']), axis=1)
df_regions_zip['city'] = df_regions_zip.apply(lambda x: check_city_in_str(x['region']), axis=1)
df_regions_zip['metro'] = df_regions_zip.apply(lambda x: check_metro_in_str(x['region']), axis=1)
df_regions_zip.head()

,region_id,region_type,region,region_str_len,zip_code,state,county,city,metro
None,,,,,,,,,
0,99999,zip,98847; WA; Wenatchee; Chelan County; Peshastin,5,98847,WA,Chelan County,Peshastin,Wenatchee
1,99998,zip,98846; WA; Okanogan County; Pateros,4,98846,WA,Okanogan County,Pateros,None
2,99997,zip,98845; WA; Wenatchee; Douglas County; Palisades,5,98845,WA,Douglas County,Palisades,Wenatchee
3,99996,zip,98844; WA; Okanogan County; Oroville,4,98844,WA,Okanogan County,Oroville,None
4,99995,zip,98843; WA; Wenatchee; Douglas County; Orondo,5,98843,WA,Douglas County,Orondo,Wenatchee


### <font color="green">3. Data</font>
[Nasdaq Data Documentation](https://data.nasdaq.com/databases/ZILLOW/data)

In [18]:
# view specific region
df_regions_zip.loc[
  (df_regions_zip['state'] == 'NJ') & 
  (df_regions_zip['city'].str.contains('South A'))]

,region_id,region_type,region,region_str_len,zip_code,state,county,city,metro
None,,,,,,,,,
34500,61233,zip,08879; NJ; New York-Newark-Jersey City; Middlesex County; South Amboy,5,08879,NJ,Middlesex County,South Amboy,New York-Newark-Jersey City


In [19]:
# median home price
# ZALL = ZHVI All Homes (SFR, Condo/Co-op) Time Series ($)	
quandl.get_table('ZILLOW/DATA', indicator_id='ZALL', region_id='61233').head()

,indicator_id,region_id,date,value
None,,,,
0,ZALL,61233,2021-07-31,374475.0
1,ZALL,61233,2021-06-30,367020.0
2,ZALL,61233,2021-05-31,360297.0
3,ZALL,61233,2021-04-30,353107.0
4,ZALL,61233,2021-03-31,346471.0


In [20]:
# median rent price
# RSSA = ZORI (Smoothed, Seasonally Adjusted): All Homes Plus Multifamily Time Series ($)
quandl.get_table('ZILLOW/DATA', indicator_id='RSSA', region_id='61233').head()

,indicator_id,region_id,date,value
None,,,,
0,RSSA,61233,2021-02-28,1938.0
1,RSSA,61233,2021-01-31,1932.0
2,RSSA,61233,2020-11-30,1875.0
3,RSSA,61233,2020-10-31,1874.0
4,RSSA,61233,2020-09-30,1873.0


## <font color="blue">Case Study: Price to Rent Ratio</font>

### <font color="green">Functions</font>

In [21]:
def get_latest_zillow_data_by_region(list_of_regions, indicator_id):
  # handle issue where api times out after 200 regions
  if len(list_of_regions) >= 200:
    temp_df_list = []
    for i in range(0, len(list_of_regions), 200):
      _df = quandl.get_table('ZILLOW/DATA', indicator_id=indicator_id, region_id=list_of_regions[i:i+200])
      temp_df_list.append(_df)
    df = pd.concat(temp_df_list)
  else:
    df = quandl.get_table('ZILLOW/DATA', indicator_id=indicator_id, region_id=list_of_regions)

  # max date per region_id
  df_max_dt = df\
    .groupby(['region_id'])['date'].max().reset_index()

  # filter down on max date per region
  return pd.merge(
      df, 
      df_max_dt, 
      how='inner', 
      on=['region_id', 'date'])

### <font color="green">1. Select Region</font>

In [22]:
# filter on state
df_state = df_regions_zip.loc[(df_regions_zip['state'] == 'NJ')]

# view top 5 counties
df_state\
.groupby(['metro'])\
.agg({'zip_code': 'count'})\
.rename(columns={'zip_code': 'num_zip_codes'})\
.sort_values(by=['num_zip_codes'], ascending=False).head()

,num_zip_codes
metro,
New York-Newark-Jersey City,353
Philadelphia-Camden-Wilmington,97
Atlantic City-Hammonton,25
Trenton,22
Ocean City,19


### <font color="green">2. Get Price & Rent Data</font>

In [23]:
# filter on region
df_filter = df_state.loc[df_state['metro'] == 'New York-Newark-Jersey City']
list_of_regions = df_filter['region_id'].tolist() # get ids to list

In [24]:
# get region median home price  
df_region_price = get_latest_zillow_data_by_region(list_of_regions, indicator_id='ZALL')
df_region_price.head(1)

,indicator_id,region_id,date,value
0,ZALL,61248,2021-07-31,412675.0


In [25]:
# get region median rent
df_region_rent = get_latest_zillow_data_by_region(list_of_regions, indicator_id='RSSA')
df_region_rent.head(1)

,indicator_id,region_id,date,value
0,RSSA,61245,2021-05-31,2031.0


### <font color="green">3. Merge Price & Rent</font>

In [26]:
# merge price and rent tables
_df_merge = pd.merge(
    df_region_price.rename(columns={'value': 'median_price'}),
    df_region_rent[['region_id', 'value']].rename(columns={'value': 'median_rent'}),
    how='left',
    on=['region_id'])
# merge with region detail
df_merge = pd.merge(
    _df_merge, 
    df_regions_zip[['region_id', 'zip_code', 'state', 'county', 'city', 'metro']], 
    how='left', 
    on=['region_id'])
df_merge.head(1)

,indicator_id,region_id,date,median_price,median_rent,zip_code,state,county,city,metro
0,ZALL,61248,2021-07-31,412675.0,NaN,08904,NJ,Middlesex County,Highland Park,New York-Newark-Jersey City


### <font color="green">4. Add Features</font>

In [27]:
# add features
df_merge['PRR'] = df_merge.apply(lambda x: x['median_price'] / (x['median_rent'] * 12), axis=1)
df_merge['lat'] = df_merge.apply(lambda x: (nomi.query_postal_code(x['zip_code']).latitude), axis=1)
df_merge['long'] = df_merge.apply(lambda x: (nomi.query_postal_code(x['zip_code']).longitude), axis=1)
# select columns
df_merge = df_merge[['region_id', 'zip_code', 'state', 'county', 'city', 
                     'metro', 'lat', 'long','median_price', 'median_rent','PRR']]
# view top price to rent ratio
df_merge.sort_values(by='PRR', ascending=False).head(1)

,region_id,zip_code,state,county,city,metro,lat,long,median_price,median_rent,PRR
23,61208,08844,NJ,Somerset County,Hillsborough Township,New York-Newark-Jersey City,40.4775,-74.6272,514138.0,1897.0,22.585574


### <font color="green">5. Visualization</font>

In [28]:
# filter only on zip codes where PRR is available
df_plot = df_merge.loc[~df_merge['PRR'].isnull()]

# set mapbox token
px.set_mapbox_access_token(mapbox_key)

# create scatter plot
fig = px.scatter_mapbox(df_plot, 
                        lat="lat", lon="long", # geography
                        color="PRR", # price to rent ratio
                        size="median_price",
                        size_max=15, 
                        zoom=8)
fig.show()

## Check for Missing Data

In [29]:
print('Num of zip codes:', len(df_merge))
print('Num of zip codes plotted:', len(df_plot))

Num of zip codes: 69
Num of zip codes plotted: 12


In [30]:
# view exampleo of missing PRR
df_merge.loc[df_merge['city'] == 'Paterson']

,region_id,zip_code,state,county,city,metro,lat,long,median_price,median_rent,PRR
56,60709,07524,NJ,Passaic County,Paterson,New York-Newark-Jersey City,40.9309,-74.1555,362820.0,NaN,NaN
57,60708,07522,NJ,Passaic County,Paterson,New York-Newark-Jersey City,40.9252,-74.1781,336641.0,NaN,NaN
58,60707,07514,NJ,Passaic County,Paterson,New York-Newark-Jersey City,40.9248,-74.1467,364553.0,NaN,NaN
59,60706,07513,NJ,Passaic County,Paterson,New York-Newark-Jersey City,40.9070,-74.1529,406440.0,NaN,NaN
63,60698,07505,NJ,Passaic County,Paterson,New York-Newark-Jersey City,40.9166,-74.1740,199555.0,NaN,NaN
64,60697,07504,NJ,Passaic County,Paterson,New York-Newark-Jersey City,40.9122,-74.1452,409988.0,NaN,NaN
65,60696,07503,NJ,Passaic County,Paterson,New York-Newark-Jersey City,40.8970,-74.1573,380423.0,NaN,NaN
66,60695,07502,NJ,Passaic County,Paterson,New York-Newark-Jersey City,40.9199,-74.1932,370768.0,NaN,NaN
67,60694,07501,NJ,Passaic County,Paterson,New York-Newark-Jersey City,40.9143,-74.1671,335848.0,NaN,NaN


In [31]:
# try to see if data is available at the CITY level
df_regions.loc[(df_regions['region_type'] == 'city') & 
               (df_regions['region'].str.contains('Paterson'))  & 
               (df_regions['region'].str.contains('NJ'))]

,region_id,region_type,region
None,,,
70997,19873,city,Paterson; NJ; New York-Newark-Jersey City; Passaic County


In [32]:
# no data for either zip code or city level
get_latest_zillow_data_by_region(['19873', '60694'], indicator_id='RSSA')

,indicator_id,value,region_id,date


# End Notebook